In [1]:
%pdb

Automatic pdb calling has been turned ON


In [2]:
from __future__ import print_function
from __future__ import division
import torch
import sys
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
from torch import autograd
from torch.autograd import Variable
import scipy.misc
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import pickle as pkl
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import tqdm
from tqdm import tqdm

PyTorch Version:  1.2.0
Torchvision Version:  0.4.0a0+6b959ee


In [3]:
torch.cuda.is_available()

True

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
data_file = "../data/train_data/train_task_1_2.csv"
batch_size = 32
lr = 0.0001
num_epochs = 100
K = 100
total_q = 28000
total_c = 4*total_q
total_s = 119000

In [7]:
df = pd.read_csv(data_file)

In [8]:
df.head()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,16997,65967,12453206,0,4,2
1,16531,62121,15686710,1,1,1
2,15911,50013,13598796,0,3,1
3,1701,104909,10511925,0,4,3
4,22896,21748,941747,0,1,4


In [9]:
df.describe()

,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
count,1.586785e+07,1.586785e+07,1.586785e+07,1.586785e+07,1.586785e+07,1.586785e+07
mean,1.380277e+04,5.952587e+04,9.916806e+06,6.429503e-01,2.502599e+00,2.488978e+00
std,7.967465e+03,3.428394e+04,5.725901e+06,4.791297e-01,1.104868e+00,1.094272e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,6.890000e+03,2.980900e+04,4.957568e+06,0.000000e+00,2.000000e+00,2.000000e+00
50%,1.383000e+04,5.958500e+04,9.916784e+06,1.000000e+00,3.000000e+00,2.000000e+00
75%,2.069400e+04,8.912500e+04,1.487596e+07,1.000000e+00,3.000000e+00,3.000000e+00
max,2.761200e+04,1.189700e+05,1.983481e+07,1.000000e+00,4.000000e+00,4.000000e+00


In [10]:
df1 = df[df['QuestionId']==16997]

In [12]:
df1.shape

(1402, 6)

In [14]:
# df1[df1['IsCorrect']==1]

In [15]:
# df['QuestionId'].max()

In [16]:
# df['UserId'].max()

In [20]:
df.iloc[100].name

100

In [23]:
def is_chosen(x,length) :
    if x.name < length :
        if x['AnswerValue'] == 1 :
            return 1
        return -1
    elif x.name < 2*length :
        if x['AnswerValue'] == 2 :
            return 1
        return -1
    elif x.name < 3*length :
        if x['AnswerValue'] == 3 :
            return 1
        return -1
    else :
        if x['AnswerValue'] == 4 :
            return 1
        return -1

In [24]:
class Question_Ans_2(Dataset):
    def __init__(self, filename, mode='train'):
        
        temp_df = pd.read_csv(filename)
        self.df = pd.concat([temp_df.copy(),temp_df.copy(),temp_df.copy(),temp_df.copy()],axis=0).reset_index()
        
        self.df['IsChoice'] = self.df.apply(lambda row : is_chosen(row,len(temp_df)),axis=1)
        
        self.df['AnsId'] = 4*(self.df['QuestionId']) + (self.df['AnswerValue'] - 1)
        
        self.ansid = self.df['AnsId'].values
        self.userid = self.df['UserId'].values
        self.ans = self.df['IsChoice'].values
                
        self.length=len(self.ans)
        
        
        if(mode=='train'):
            start=int(0*self.length)
            end=int(0.8*self.length)
        else:
            start=int(0.8*self.length)
            end=int(1*self.length)
            
            
        self.ansid = self.ansid[start:end]
        self.userid = self.userid[start:end]
        self.ans = self.ans[start:end]
        
        
        
        self.length=len(self.ans)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        
        aid = self.ansid[idx]
        uid = self.userid[idx]
        ans = self.ans[idx]
        return aid,uid,ans

In [25]:
train_dataset = Question_Ans_2(filename=data_file,mode='train')
val_dataset = Question_Ans_2(filename=data_file,mode='val')

In [27]:
train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset,batch_size=batch_size, shuffle=False)

In [28]:
dataloader = {}
dataloader['train'] = train_dataloader
dataloader['val'] = val_dataloader

In [15]:
dtype=torch.FloatTensor

In [16]:
# Declare two matrices 
# Q = shape (total_q,K)
# U = shape (total_s,K)
C = torch.nn.Embedding(total_c,K)
U = torch.nn.Embedding(total_s,K)

C = C.to(device)
U = U.to(device)

In [17]:
# embedding = torch.nn.Embedding(5,8)

In [18]:
# embedding(torch.LongTensor(np.array([3,4,3])))

In [19]:
def get_cvector(choices):
    
    ans = C(torch.LongTensor(choices).to(device))
    
    ans = ans.to(device)
    return ans

In [20]:
def get_uvector(users):
    
    ans = U(torch.LongTensor(users).to(device))
    ans = ans.to(device)
    return ans

In [21]:
test1 = torch.randn(32,100)
test2 = torch.randn(32,100)
test1 = torch.unsqueeze(test1,1)
test2 = torch.unsqueeze(test2,2)
print(test1.shape)
print(test2.shape)
res = torch.bmm(test1,test2)
print(res.shape)
res = torch.squeeze(res)
print(res.shape)

torch.Size([32, 1, 100])
torch.Size([32, 100, 1])
torch.Size([32, 1, 1])
torch.Size([32])


In [22]:
def get_score(cvectors,uvectors):
    
    c_unsq = torch.unsqueeze(cvectors, 1)
    u_unsq = torch.unsqueeze(uvectors, 2)
    score = torch.bmm(c_unsq,u_unsq)
    score = torch.squeeze(score)
    return score

In [23]:
params_to_update = list(C.parameters()) + list(U.parameters())

In [24]:
criterion = nn.MSELoss()
optimizer = optim.SGD(params=params_to_update,lr=lr, momentum=0.9)

In [ ]:
best_loss = 1e9
for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    print('-' * 10)
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        
            
        running_loss = 0.0
        
        # Iterate over data.
        cnt = 0
#         print("*********** entered ",phase,"*********************")
        for choices,users,ans in tqdm(dataloader[phase]):
            
#             print("************ loaded data of one batch*******")
            
            cnt = cnt + 1
            ans = torch.tensor(ans)
            ans = ans.type(dtype)
            ans = ans.to(device)
            
            cvectors = get_qvector(choices)
            uvectors = get_uvector(users)
            
            
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward
            
            scores = get_score(cvectors,uvectors)
            
            loss = criterion(scores,ans)
            
            if phase == 'train':
                loss.backward()
                optimizer.step()
        
            # statistics
            running_loss += loss.item()
            
            # compute correct
            # running_corrects += torch.sum(preds == labels.data)
            
            if (cnt % 1000 == 0 ) :
                print('Batch {} {} Loss: {:.4f} '.format(str(cnt),phase, running_loss/(cnt*batch_size)))
            

        epoch_loss = running_loss / len(dataloader[phase].dataset)

        print(' {} Loss: {:.4f} '.format(phase, epoch_loss))
        
        # deep copy the model
        if phase == 'val' and epoch_loss < best_loss:
            best_loss = epoch_loss
            
            c_file = '../Weights/simple_fact_q' + '_best' + '.pkl'
            u_file = '../Weights/simple_fact_u' + '_best' + '.pkl'
            q_f = open(q_file, 'wb')
            u_f = open(u_file, 'wb')
            pickle.dump(Q,q_f)
            pickle.dump(U,u_f)
        
        q_file = '../Weights/simple_fact_q_' + str(epoch+1) + '.pkl'
        u_file = '../Weights/simple_fact_u_' + str(epoch+1) + '.pkl'
        q_f = open(q_file, 'wb')
        u_f = open(u_file, 'wb')
        pickle.dump(Q,q_f)
        pickle.dump(U,u_f)
    print()
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
print('Best val Loss: {:4f}'.format(best_loss))

  0%|          | 0/396697 [00:00<?, ?it/s]

Epoch 0/4
----------


/home/apps/anaconda3/5.2.0/gnu/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  0%|          | 1030/396697 [00:07<34:14, 192.63it/s] 

Batch 1000 train Loss: 3.1129 


  1%|          | 2020/396697 [00:12<35:55, 183.06it/s]

...

100%|█████████▉| 396031/396697 [36:13<00:03, 182.24it/s]

Batch 396000 train Loss: 2.9304 


100%|██████████| 396697/396697 [36:17<00:00, 182.18it/s]


NameError: name 'dataloaders' is not defined

> <ipython-input-25-ea5c7fa2ab7a>(49)<module>()
     47 
     48 
---> 49         epoch_loss = running_loss / len(dataloaders[phase].dataset)
     50 
     51         print(' {} Loss: {:.4f} '.format(phase, epoch_loss))

